In [39]:
from langchain_google_vertexai import ChatVertexAI
import os
import json
import re

from typing import Set

import random

## Загрузка датасета

In [2]:
PATH = "..\\src\\data\\komus\\dataset.json"

In [3]:
with open(PATH, "r", encoding="UTF-8") as file:
    data = json.load(file)

In [4]:
print(len(data))

83755


## Определение параметров

In [8]:
PROJECT_ID = 'axial-chemist-425510-p2'
LOCATION = "europe-west4"

# Укажите путь к вашему JSON-файлу с кредами
SERVICE_ACCOUNT_FILE = "../secrets/axial-chemist-425510-p2-b4eb1d622fbe.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = SERVICE_ACCOUNT_FILE


llm_params = {
    'model_name': 'gemini-2.0-flash-001',
    'project': PROJECT_ID,
    'location': LOCATION
}

In [37]:
llm = ChatVertexAI(**llm_params)

In [ ]:
product = data[41412]

PROMPT = f"""
Ты экспертная система по определению соответствия товарной позиции и ее разложения.
Пример работы:
1) Товарная позиция: Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе).
Разложение: "Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия"
Ответ: ["Торговая марка", "Наличие ластика", "Твердость грифеля", "Заточенный", "Профиль карандаша", "Цвет корпуса", "Кол-во штук в единице продажи"]
2) Товарная позиция: Сабо женские EVART Ола ЭВА серые (размер 39).
Разложение: "Торговая марка": "Evart", "Коллекция": "Сабо ЭВА Ола", "Тип": "сабо", "Модель": "Ола", "Материал верха обуви": "ЭВА", "Пол": "женский", "Размер": "39", "Цвет": "серый", "Материал подошвы": "ЭВА", "Материал стельки": "отсутствует", "Стелька": "отсутствует", "Подносок": "нет", "Защитные свойства": "от общих загрязнений", "Метод крепления подошвы": "литьевой", "С ремешком": "да", "Перфорация": "Да", "Тип подошвы": "плоская", "Сфера применения": "медицина , пищевое производство и общественное питание , хорека", "Стандарты": "ТР ТС 017/2011", "Честный знак": "Да", "Количество единиц продаж в транспортной упаковке": "1", "Страна происхождения": "Россия", "Вес": "0.227 кг"
Ответ: ["Торговая марка", "Тип", "Модель", "Материал верха обуви", "Пол", "Размер", "Цвет", "Материал подошвы"]
3) Товарная позиция: Бумага для цветной лазерной печати Xerox Colotech + ( A4, 250 г/кв.м, 250 листов, 003R94671)
Разложение: "Торговая марка": "Xerox Colotech +", "Покрытие": "без покрытия (каландрированная бумага)", "Плотность листа бумаги": "250 г/кв.м", "Белизна CIE": "161 ± 3%", "Формат листов": "А4", "Сертификация по экологическим стандартам": "Нет", "Яркость бумаги": "110", "Количество единиц продаж в транспортной упаковке": "144", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "4", "Страна происхождения": "Австрия", "Количество листов в упаковке": "250 шт"
Ответ: ["Торговая марка", "Плотность листа бумаги", "Формат листов", "Количество листов в упаковке"]

Дай ответ на данную позицию {product["title"]} и ее разложение {json.dumps(product["attributes"], ensure_ascii=False)}
""".strip()
PROMPT

'Ты экспертная система по определению соответствия товарной позиции и ее разложения.\nПример работы:\n1) Товарная позиция: Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе).\nРазложение: "Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия"\nОтвет: ["Торговая марка", "Наличие ластика", "Твердость грифеля", "Заточенный", "Профиль карандаша", "Цвет корпуса", "Кол-во штук в единице продажи"]\n2) Товарная позиция: Сабо женские EVART Ола ЭВА серые (размер 39).\nРазложение: "Торговая марка": "Evart", "Коллекция": "Сабо ЭВА О

In [38]:
res = llm.invoke(PROMPT)

res_text = res.content

res_text

'["Торговая марка", "Вид товара", "Материал", "Объем полный", "Объем рабочий", "Цвет", "Кол-во штук в единице продажи"]\n'

In [36]:
print(json.dumps(product["attributes"], ensure_ascii=False))
product


{"Торговая марка": "Комус", "Коллекция": "Одноразовая посуда Комус", "Вид товара": "стакан", "Материал": "бумага", "Объем полный": "430 мл", "Объем рабочий": "300 мл", "Назначение": "для холодных напитков , для горячих напитков", "Особенности": "экологически безопасный товар", "Цвет": "красный", "Рисунок на корпусе": "Нет", "Сырье": "первичное", "Температура воды максимальная": "90 град", "Количество слоев": "1", "Ценовая категория": "эконом", "Кол-во штук в единице продажи": "50", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "отсутствует", "Количество единиц продаж в транспортной упаковке": "16", "Страна происхождения": "Россия", "Диаметр": "90 мм"}


{'link': 'https://www.komus.ru/katalog/posuda-i-tekstil/odnorazovaya-posuda-i-upakovka-/stakany-odnorazovye/stakan-odnorazovyj-bumazhnyj-komus-430-300-ml-krasnyj-50-shtuk-v-upakovke-/p/1033896/',
 'title': 'Стакан одноразовый бумажный Комус 430/300 мл красный (50 штук в упаковке)',
 'attributes': {'Торговая марка': 'Комус',
  'Коллекция': 'Одноразовая посуда Комус',
  'Вид товара': 'стакан',
  'Материал': 'бумага',
  'Объем полный': '430 мл',
  'Объем рабочий': '300 мл',
  'Назначение': 'для холодных напитков , для горячих напитков',
  'Особенности': 'экологически безопасный товар',
  'Цвет': 'красный',
  'Рисунок на корпусе': 'Нет',
  'Сырье': 'первичное',
  'Температура воды максимальная': '90 град',
  'Количество слоев': '1',
  'Ценовая категория': 'эконом',
  'Кол-во штук в единице продажи': '50',
  'Кол-во единиц продаж в промежуточной упаковке (кратность)': 'отсутствует',
  'Количество единиц продаж в транспортной упаковке': '16',
  'Страна происхождения': 'Россия',
  'Диаметр': 

## Случайный выбор из подборки

In [50]:
idx = random.choices(list(range(len(data))), k = 5)
idx

[6515, 4667, 69498, 9565, 79877]

In [56]:
llm = ChatVertexAI(**llm_params)

SYSTEM_PROMPT = """
Ты экспертная система по определению соответствия товарной позиции и ее разложения.
Пример работы:
1) Товарная позиция: Карандаш чернографитный пластиковый НВ с ластиком Выбор есть заточенный шестигранный черный корпус (12 штук в наборе).
Разложение: "Торговая марка": "Выбор есть", "Твердость грифеля": "HB", "Наличие ластика": "Да", "Заточенный": "Да", "Профиль карандаша": "шестигранный", "Материал корпуса": "пластик", "Цвет корпуса": "черный", "Длина корпуса карандаша": "185 мм", "Единица продажи": "упаковка", "Кол-во штук в единице продажи": "12", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "6", "Количество единиц продаж в транспортной упаковке": "90", "Страна происхождения": "Россия"
Ответ: ["Торговая марка", "Наличие ластика", "Твердость грифеля", "Заточенный", "Профиль карандаша", "Цвет корпуса", "Кол-во штук в единице продажи"]
2) Товарная позиция: Сабо женские EVART Ола ЭВА серые (размер 39).
Разложение: "Торговая марка": "Evart", "Коллекция": "Сабо ЭВА Ола", "Тип": "сабо", "Модель": "Ола", "Материал верха обуви": "ЭВА", "Пол": "женский", "Размер": "39", "Цвет": "серый", "Материал подошвы": "ЭВА", "Материал стельки": "отсутствует", "Стелька": "отсутствует", "Подносок": "нет", "Защитные свойства": "от общих загрязнений", "Метод крепления подошвы": "литьевой", "С ремешком": "да", "Перфорация": "Да", "Тип подошвы": "плоская", "Сфера применения": "медицина , пищевое производство и общественное питание , хорека", "Стандарты": "ТР ТС 017/2011", "Честный знак": "Да", "Количество единиц продаж в транспортной упаковке": "1", "Страна происхождения": "Россия", "Вес": "0.227 кг"
Ответ: ["Торговая марка", "Тип", "Модель", "Материал верха обуви", "Пол", "Размер", "Цвет", "Материал подошвы"]
3) Товарная позиция: Бумага для цветной лазерной печати Xerox Colotech + ( A4, 250 г/кв.м, 250 листов, 003R94671)
Разложение: "Торговая марка": "Xerox Colotech +", "Покрытие": "без покрытия (каландрированная бумага)", "Плотность листа бумаги": "250 г/кв.м", "Белизна CIE": "161 ± 3%", "Формат листов": "А4", "Сертификация по экологическим стандартам": "Нет", "Яркость бумаги": "110", "Количество единиц продаж в транспортной упаковке": "144", "Кол-во единиц продаж в промежуточной упаковке (кратность)": "4", "Страна происхождения": "Австрия", "Количество листов в упаковке": "250 шт"
Ответ: ["Торговая марка", "Плотность листа бумаги", "Формат листов", "Количество листов в упаковке"]
""".strip()

USER_PROMPT = """Дай ответ на данную позицию {0} и ее разложение {1}.""".strip()

llm.invoke(SYSTEM_PROMPT)

for id in idx[:2]:
    product = data[id]

    prompt = USER_PROMPT.format(product["title"], json.dumps(product["attributes"], ensure_ascii=False))

    print(prompt)

    res = llm.invoke(prompt)

    print(json.dumps(product, ensure_ascii=False, indent=4))
    print(f"Ответ: {res.content}")
    

Дай ответ на данную позицию Чай черный Tess Pleasure 100 пакетиков (яблоко, шиповник) и ее разложение {"Торговая марка": "Tess", "Тип чая": "черный", "Вкус/добавки": "яблоко, шиповник", "Тип пакетика": "стандартный", "Наличие индивидуального пакетика": "Да", "Чайных пакетиков в упаковке": "100", "Тип упаковки": "картонная коробка", "Количество единиц продаж в транспортной упаковке": "9", "Подарочный набор": "Нет", "Страна происхождения": "Россия", "Срок хранения от даты производства (месяцев)": "36", "Вес": "150 г"}.
{
    "link": "https://www.komus.ru/katalog/produkty-pitaniya/chaj/chaj-paketirovannyj/chaj-chernyj-paketirovannyj/chaj-chernyj-tess-pleasure-100-paketikov-yabloko-shipovnik-/p/730056/",
    "title": "Чай черный Tess Pleasure 100 пакетиков (яблоко, шиповник)",
    "attributes": {
        "Торговая марка": "Tess",
        "Тип чая": "черный",
        "Вкус/добавки": "яблоко, шиповник",
        "Тип пакетика": "стандартный",
        "Наличие индивидуального пакетика": "Да",
